In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'talk.politics.guns',
              'sci.space']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

print(train.data[0])


From: fcrary@ucsu.Colorado.EDU (Frank Crary)
Subject: Re: Riddle me this...
Nntp-Posting-Host: ucsu.colorado.edu
Organization: University of Colorado, Boulder
Distribution: usa
Lines: 16

In article <1r1lp1INN752@mojo.eng.umd.edu> chuck@eng.umd.edu (Chuck Harris - WA3UQV) writes:
>>If so, why was CS often employed against tunnels in Vietnam?

>CS "tear-gas" was used in Vietnam because it makes you wretch so hard that
>your stomach comes out thru your throat.  Well, not quite that bad, but
>you can't really do much to defend yourself while you are blowing cookies.

I think the is BZ gas, not CS or CN. BZ gas exposure results in projectile
vomiting, loss of essentially all muscle control, inability to concentrate
or think rationally and fatal reactions in a significant fraction of
the population. For that reason its use is limited to military
applications.

                                                          Frank Crary
                                                          CU B

In [3]:
y_train = train.target
print(y_train)

def prob_of_class(target_list, num_of_classes):
    total = len(target_list)
    probabilities = [0 for _ in range(num_of_classes)]

    for y in target_list:
        probabilities[y] += 1
    probabilities = [prob/total for prob in probabilities]

    return probabilities

prob_class = prob_of_class(y_train, len(train.target_names))
print(f' probabilities are {prob_class},\n in total their sum is {sum(prob_class)}')

[2 2 1 ... 1 2 2]
 probabilities are [0.2964793082149475, 0.3662754786905497, 0.3372452130945028],
 in total their sum is 1.0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english", min_df=5,token_pattern="[^\W\d_]+", binary=True)
D = vectorizer.fit_transform(train.data)
D_test = vectorizer.transform(test.data)

vectorizer.get_feature_names_out()

array(['aa', 'aario', 'aaron', ..., 'zoology', 'zv', 'ÿ'], dtype=object)

In [126]:
naive_index = np.where(vectorizer.get_feature_names_out() == 'naive')[0][0]
alpha = 10 ** -5

def log_prob_laplace(word_matrix, alpha, num_of_classes, target_list, key_word_index):
    total_key_words = [0 for _ in range(num_of_classes)]
    total_words_classes = [0 for _ in range(num_of_classes)]
    total_words_document = word_matrix.sum(axis=1).getA().flatten()

    for i, y in enumerate(target_list):
        total_key_words[y] += word_matrix[i, key_word_index]
        total_words_classes[y] += total_words_document[i]
    
    probabilities = [0 for _ in range(num_of_classes)]
    for y in range(num_of_classes):
        probabilities[y] = np.log((total_key_words[y] + alpha) / (total_words_document[y] + alpha * sum(total_words_document)))

    return probabilities

print(log_prob_laplace(D, alpha, 3, y_train, naive_index))

[-2.606241954135954, -4.946906632387673, -2.7203111961144097]


In [127]:
num_documents = len(y_train)
num_words = len(vectorizer.get_feature_names_out())

first_doc = D[0, :]

words_class_log_prob = [log_prob_laplace(D, alpha, 3, y_train, word_index) for word_index in range(num_words)]
print(words_class_log_prob)
print(words_class_log_prob[naive_index])

for doc_index, y in enumerate(y_train):
    pass

[[-3.522529686020609, -3.5606197712209076, -3.007992435235288], [-2.606241954135954, -16.459842097307902, -15.619533522201404], [-4.215671866618054, -16.459842097307902, -1.0620851433173897], [-15.728607331538283, -3.1551554964443405, -2.3148469213378426], [-4.215671866618054, -4.946906632387673, -2.3148469213378426], [-3.522529686020609, -4.946906632387673, -2.4971681447990752], [-15.728607331538283, -16.459842097307902, -0.7054103422357424], [-2.8293850054512886, -16.459842097307902, -4.106598057281175], [-1.3253095531164882, -3.1551554964443405, -1.4675500133304578], [-0.5521199640281841, -1.258036928223768, -0.02907044383394491], [-2.018456178121341, -4.253764451790228, -15.619533522201404], [-3.522529686020609, -3.337476719905573, -15.619533522201404], [-15.728607331538283, -16.459842097307902, -2.160696479605454], [-4.215671866618054, -3.848301010341786, -4.106598057281175], [-2.8293850054512886, -16.459842097307902, -2.7203111961144097], [-3.522529686020609, -16.459842097307902,